In [1]:
import numpy as np
import scipy

In [10]:
from scipy.linalg import eigh

# Example matrices
A = np.array([
  [4, 1], 
  [1, 3]]
)
B = np.array([
  [2, .5], 
  [0.5, 1.5]])

# Solve the generalized eigenvalue problem
eigenvalues, eigenvectors = eigh(A, B, lower=False, type=1)

print("Eigenvalues:", eigenvalues)
print("Eigenvectors:", eigenvectors)

Eigenvalues: [2. 2.]
Eigenvectors: [[ 0.70710678 -0.21320072]
 [ 0.          0.85280287]]


In [14]:
A @ eigenvectors - eigenvalues * B @ eigenvectors

array([[ 4.44089210e-16, -2.22044605e-16],
       [ 1.11022302e-16,  0.00000000e+00]])

In [23]:
import numpy as np
from scipy.linalg import cholesky, solve_triangular, eigh

# Input matrices
A = np.array([[4.0, 1.0],
              [1.0, 3.0]])
B = np.array([[2.0, 0.5],
              [0.5, 1.5]])

print("Original A:")
print(A)
print("\nOriginal B:")
print(B)

# Step 1: Cholesky factorization of B
U = cholesky(B, lower=False)  # Upper triangular Cholesky factor
print("\nCholesky factor U (B = Uᵀ * U):")
print(U)

# Step 2: Transform A into the standard eigenvalue problem
C = np.zeros_like(A)
for i in range(A.shape[0]):
    for j in range(i, A.shape[1]):  # Only compute upper triangular part
        sum_ij = 0.0
        for k in range(A.shape[0]):
            sum_ij += A[i, k] / U[k, k] / U[k, k] * A[j, k]
        C[i, j] = sum_ij
        C[j, i] = sum_ij  # Enforce symmetry
print("\nTransformed matrix C (C = inv(U) * A * inv(Uᵀ)):")
print(C)

# Symmetry check for C
print("\nSymmetry check for C (C - C.T):")
print(C - C.T)

# Step 3: Compute eigenvalues and eigenvectors of C
w, Y = eigh(C)
print("\nEigenvalues of C:")
print(w)
print("\nEigenvectors of C:")
print(Y)

# Step 4: Backtransform eigenvectors to the original problem
X = solve_triangular(U, Y, lower=False, trans='T', overwrite_b=False)
for i in range(X.shape[1]):
    norm = np.linalg.norm(X[:, i])
    X[:, i] /= norm
print("\nBacktransformed eigenvectors (original problem):")
print(X)

# Orthonormality check
print("\nOrthonormality check for backtransformed eigenvectors (Xᵀ * X):")
print(np.dot(X.T, X))

# Correct the eigenvalues
w_corrected = w / np.diag(U) ** 2
print("\nCorrected Eigenvalues (original problem):")
print(w_corrected)

# Compare with SciPy's generalized eigenvalue solver
w_scipy, v_scipy = eigh(A, B)
print("\nSciPy Eigenvalues:")
print(w_scipy)
print("\nSciPy Eigenvectors:")
print(v_scipy)

Original A:
[[4. 1.]
 [1. 3.]]

Original B:
[[2.  0.5]
 [0.5 1.5]]

Cholesky factor U (B = Uᵀ * U):
[[1.41421356 0.35355339]
 [0.         1.17260394]]

Transformed matrix C (C = inv(U) * A * inv(Uᵀ)):
[[8.72727273 4.18181818]
 [4.18181818 7.04545455]]

Symmetry check for C (C - C.T):
[[0. 0.]
 [0. 0.]]

Eigenvalues of C:
[ 3.62083537 12.1518919 ]

Eigenvectors of C:
[[ 0.63358477 -0.77367327]
 [-0.77367327 -0.63358477]]

Backtransformed eigenvectors (original problem):
[[ 0.49100774 -0.82455886]
 [-0.87115521 -0.56577618]]

Orthonormality check for backtransformed eigenvectors (Xᵀ * X):
[[1.         0.08801408]
 [0.08801408 1.        ]]

Corrected Eigenvalues (original problem):
[1.81041768 8.83773957]

SciPy Eigenvalues:
[2. 2.]

SciPy Eigenvectors:
[[ 0.70710678 -0.21320072]
 [ 0.          0.85280287]]


In [29]:
def forward_substitution(L, b):
    """Solve L x = b for x (L is lower triangular)."""
    n = L.shape[0]
    x = np.zeros_like(b)
    for i in range(n):
        x[i] = (b[i] - np.dot(L[i, :i], x[:i])) / L[i, i]
    return x

def backward_substitution(U, b):
    """Solve U x = b for x (U is upper triangular)."""
    n = U.shape[0]
    x = np.zeros_like(b)
    for i in reversed(range(n)):
        x[i] = (b[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x

def mat_mult_loops(A, B):
    """Basic matrix multiplication A @ B using loops."""
    n, m = A.shape
    m2 = B.shape[1]
    result = np.zeros((n, m2))
    for i in range(n):
        for j in range(m2):
            for k in range(m):
                result[i, j] += A[i, k] * B[k, j]
    return result

def sygv_numpy_expanded(A, B):
    """
    Tutorial-style: Solve A x = λ B x using low-level linear algebra.
    """
    n = A.shape[0]
    
    # Step 1: Cholesky factorization B = L L^T
    L = np.linalg.cholesky(B)

    # Step 2: Solve L Y = A for Y => column by column
    Y = np.zeros_like(A)
    for j in range(n):
        Y[:, j] = forward_substitution(L, A[:, j])
    
    # Step 3: Solve L^T C^T = Y^T to get C^T => again, column by column
    C_T = np.zeros_like(A)
    for j in range(n):
        C_T[:, j] = backward_substitution(L.T, Y.T[:, j])
    C = C_T.T

    # Step 4: Solve standard eigenproblem C y = λ y
    w, y = np.linalg.eigh(C)

    # Step 5: Backsolve to get original eigenvectors: L^T x = y => x = (L^T)^{-1} y
    x = np.zeros_like(y)
    for j in range(n):
        x[:, j] = backward_substitution(L.T, y[:, j])

    return w, x


In [36]:
import numpy as np

def forward_substitution(L, b):
    """Solve L x = b for x where L is lower-triangular."""
    n = L.shape[0]
    x = np.zeros_like(b)
    for i in range(n):
        s = 0.0
        for j in range(i):
            s += L[i, j] * x[j]
        x[i] = (b[i] - s) / L[i, i]
    return x

def backward_substitution(U, b):
    """Solve U x = b for x where U is upper-triangular."""
    n = U.shape[0]
    x = np.zeros_like(b)
    for i in reversed(range(n)):
        s = 0.0
        for j in range(i + 1, n):
            s += U[i, j] * x[j]
        x[i] = (b[i] - s) / U[i, i]
    return x

def sygv_numpy_expanded_correct(A, B):
    """
    Tutorial-style generalized eigenvalue solver using low-level operations.
    Solves A x = λ B x.
    """
    n = A.shape[0]

    # Step 1: Cholesky decomposition: B = L L^T
    L = np.linalg.cholesky(B)

    # Step 2: Compute Y = L^{-1} A using forward substitution (column-by-column)
    Y = np.zeros_like(A)
    for j in range(n):
        Y[:, j] = forward_substitution(L, A[:, j])

    # Step 3: Compute C = Y @ L^{-T} = (solve L^T, each row of Y.T).T
    C = np.zeros_like(A)
    for j in range(n):
        C[j, :] = backward_substitution(L.T, Y[j, :])

    # Step 4: Solve the standard symmetric eigenproblem: C y = λ y
    w, y = np.linalg.eigh(C)

    # Step 5: Recover original eigenvectors x = L^{-T} y
    x = np.zeros_like(y)
    for j in range(n):
        x[:, j] = backward_substitution(L.T, y[:, j])

    return w, x


In [37]:
# Example
A = np.array([[5, 4], [4, 5]], dtype=float)
B = np.array([[4, 1], [1, 3]], dtype=float)

w, v = sygv_numpy(A, B)

print("Eigenvalues:", w)
print("Eigenvectors:\n", v)


Eigenvalues: [0.39780511 2.05674035]
Eigenvectors:
 [[ 0.456727   -0.25323451]
 [-0.43220362 -0.4205192 ]]


In [39]:
# Example
A = np.array([[5, 4], [4, 5]], dtype=float)
B = np.array([[4, 1], [1, 3]], dtype=float)

w, v = sygv_numpy_expanded_correct(A, B)

print("Eigenvalues:", w)
print("Eigenvectors:\n", v)

Eigenvalues: [0.67167549 1.73135862]
Eigenvectors:
 [[-0.50681194  0.12597193]
 [ 0.30820453  0.51831104]]


In [26]:
import scipy.linalg


scipy.linalg.eigh(A, B)

(array([0.39780511, 2.05674035]),
 array([[ 0.456727  , -0.25323451],
        [-0.43220362, -0.4205192 ]]))

In [17]:
A @ X, w * B @ X

(array([[-1.39619104,  2.37009124],
        [ 1.64255462,  1.96265783]]),
 array([[-0.47920744,  1.86337709],
        [ 2.40190304,  2.20066656]]))